***

## CSC424-202: Notes on RSA

author: burt rosenberg
<br>
date: 1 may 2020
last upate: 4 may 2020

***


In [1]:
import math
import random


class Nthy_U:
    
    @staticmethod
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = Nthy_U.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    @staticmethod
    def modinv(a, m):
        g, x, y = Nthy_U.egcd(a, m)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % m

    @staticmethod
    def is_prime(n):
        if n < 2: return False
        if n % 2 == 0: return False
        k = 3
        while k*k <= n:
            if n % k == 0: return False
            k += 2
        return True

class RSA_publickey:
    
    def __init__(self, p, q, encryption_exponent):
        assert(Nthy_U.is_prime(p) and Nthy_U.is_prime(q))
        assert(p!=q)
        self.n = p*q
        self.phi_n = (p-1)*(q-1)
        self.e_exp = encryption_exponent
        assert( math.gcd(self.e_exp,self.phi_n) )
        self.d_exp = Nthy_U.modinv(self.e_exp,self.phi_n)
        assert ( (self.e_exp*self.d_exp) % self.phi_n == 1 )
        
        bits = math.ceil(math.log(self.n+1,2))
        assert(2**bits>self.n)
        self.s_sec_bits = bits//4
        self.msg_bits = bits - self.s_sec_bits - 1

    def get_modulus(self):
        # this is public
        return self.n
    
    def get_encryption_exponent(self):
        # this is public
        return self.e_exp
    
    def get_phi_of_modulus(self):
        # this is private
        assert False
        return 0
    
    def get_decryption_exponent(self):
        # this is private
        assert False
        return 0
    
    def get_message_size(self):
        return 2**self.msg_bits-1
        
    def encrypt(self,plaintext,semantic_security=True,verbose=False):
        if not semantic_security:
            return ((plaintext**self.e_exp)%self.n)

        assert(plaintext<2**self.msg_bits)
        b = 2**self.s_sec_bits
        r = random.randint(b//2,b-1)
        wrap_m = (r << self.msg_bits) | plaintext
        assert(wrap_m<self.n)
        
        if math.gcd(wrap_m,self.n)!=1:
            print('factored the modulus, prime is {}'.format(math.gcd(wrap_m,self.n)))
            return 0  
        if verbose: print('rand|msg = {}|{}, rand={:#b}, msg={:#b}, wraped={:#b}'.format( 
            self.s_sec_bits,self.msg_bits,r,plaintext,wrap_m))
        
        return ((wrap_m**self.e_exp)%self.n)
    
    def decrypt(self,ciphertext,semantic_security=True,verbose=False):
        if not semantic_security:
            return ((cipertext**self.e_exp)%self.n)
        
        wrap_m = (ciphertext**self.d_exp)%self.n
        if verbose: print(bin(wrap_m))
        m = ((wrap_m>>self.msg_bits)<<self.msg_bits)^wrap_m
        return m    


In [2]:

rsa_instance = RSA_publickey(137,167,3)

for i in range(2000):
    plaintext = random.randint(1,rsa_instance.get_message_size())
    ciphertext = rsa_instance.encrypt(plaintext)
    if ciphertext!=0:
        decrypt_pt = rsa_instance.decrypt(ciphertext)
        assert decrypt_pt == plaintext

print("done!")

factored the modulus, prime is 137
factored the modulus, prime is 167
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 167
factored the modulus, prime is 167
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 167
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
factored the modulus, prime is 137
done!


In [3]:
# 
# a small walk through example, before we get to the
# all the software fancy stuff
#

# pick two primes, they must be different

p,q =137,167

# their product is one of two elements to the public key; never reveal p and q - in fact, erase them
# after the next step
n = p*q

# we don't need n as much as phi_n, calculated here
phi_n = (p-1)*(q-1)
# and we don't really need phi_n, what we need is a pair of numbers, private-exponent and public-exponent
# that are related by phi_n mathematically

pub_exp = 3
assert(math.gcd(3,phi_n)==1)   # we need this true 
pri_exp = Nthy_U.modinv(pub_exp,phi_n) # and we need this calculated out

# now you can erase n and phi_n. we release public-exponent and keep private-exponent private

print('public stuff:\n\tn={}\n\tpublic-exponent={}\nprivate stuff to keep\n\tprivate-exponent={}\nprivate stuff to erase\n\tp={}\n\tq={}\n\tphi_n={}'.format(n,pub_exp, pri_exp,p,q,phi_n, ))

# how to use this in a simple way

def rsa_encrypt(message,pub_exp,n):
    assert(math.gcd(message,n))
    if message>=n:
        return 0  # this is otherwise an impossible value
    return (message**pub_exp)%n

def rsa_decrypt(ciphertext,pri_exp,n):
    return (ciphertext**pri_exp)%n

#example
message = 34
print('\n\nmessage:{}->encryption:{}->decrypted:{}\n\n'.format(message,rsa_encrypt(message,pub_exp,n),
                        rsa_decrypt(rsa_encrypt(message,pub_exp,n),pri_exp,n)))


public stuff:
	n=22879
	public-exponent=3
private stuff to keep
	private-exponent=15051
private stuff to erase
	p=137
	q=167
	phi_n=22576


message:34->encryption:16425->decrypted:34




In [6]:
# 
# a small walk through example of signatures
#

def rsa_sign(message_to_sign,pri_exp,n):
    assert(math.gcd(message_to_sign,n))
    if message_to_sign>=n:
        return 0  # this is otherwise an impossible value
    return (message_to_sign,(message_to_sign**pri_exp)%n)

def rsa_verify(signed_message,pub_exp,n):
    d = (signed_message[1]**pub_exp)%n
    return signed_message[0]==d


message_to_sign = 41
signed_message = rsa_sign(message_to_sign,pri_exp,n)
print(signed_message)
print(rsa_verify(signed_message,pub_exp,n))

(41, 6038)
True
